# Data validation
This is a mostly manual section, where we check data validity and inconsistencies. Also, it is important for the data to be resilient to bugs.

In [1]:
import os
import pickle as pkl
import re
import sys
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# %matplotlib inline
warnings.filterwarnings('ignore')

In D:\Users\Marcelo\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In D:\Users\Marcelo\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In D:\Users\Marcelo\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In D:\Users\Marcelo\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In D:\Users\Marcelo\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classi

# define functions

In [2]:
# get numerical columns
def get_numerical_mask(df):
    type_mask = []
    for i in df.dtypes:
        if str(i).startswith('float') or str(i).startswith('int'): # or str(i).startswith('bool')
            type_mask.append(True)
        else: type_mask.append(False)
    num_cols = list(np.array(df.columns)[type_mask])
    other_cols = list(np.array(df.columns)[[not elem for elem in type_mask]])
    
    return num_cols, other_cols

def duplicate_cleanup(df, col_id):
    original_size = df.shape[0]
    col_subset = df.columns.to_list()
    n_duplicates = df[(df.duplicated(col_subset))].shape[0]
    df.drop_duplicates(col_subset, inplace=True)
    print('number of removed duplicates:', n_duplicates)
    
    true_duplicates = df[(df.duplicated(col_id))].shape[0]
    if true_duplicates != 0:
        raise ValueError('There are still duplicates to verify')
        
    return df

# Define paths and capture data

In [3]:
inputs = os.path.join('..', 'data', '01_raw')
outputs = os.path.join('..', 'data', '02_intermediate')
reports = os.path.join('..', 'data', '06_reporting')

data = pd.read_csv(os.path.join(inputs, 'data_sample.csv'))
# data_identity = pd.read_csv(os.path.join(inputs, 'train_identity.csv'))

In [4]:
print('shape of data:', data.shape[0])
data.head()

shape of data: 10000


,Unnamed: 0,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,499774,3486774,0,13107389,38.056,C,9633,130.0,185.0,visa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,75695,3062695,0,1650884,150.000,R,15063,514.0,150.0,visa,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,286443,3273443,0,7048761,56.500,W,9006,555.0,143.0,mastercard,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,397445,3384445,0,10011292,8.459,C,11201,103.0,185.0,visa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,502059,3489059,0,13159069,77.950,W,7919,194.0,150.0,mastercard,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# standardize data

#### columns

In [5]:
data.rename({c: c.lower() for c in data.columns}, axis=1, inplace=True)
data.rename({'isfraud': 'y', 'transactionid': 'id'}, axis=1, inplace=True)

#### strip values of categorical

# check duplicates for main data

In [6]:
data = duplicate_cleanup(data, 'id')

number of removed duplicates: 0


# set index

In [7]:
data.set_index('id', inplace=True)

# build new variables

In [8]:
# todo

# drop unused columns

In [9]:
data.drop([c for c in data.columns if c.startswith('v')], axis=1, inplace=True)
data.drop(['unnamed: 0'], axis=1, inplace=True)

# boolean features to float (0,1)

In [10]:
# bool -> int
for col in ['m1', 'm2', 'm3', 'm5', 'm6', 'm7', 'm8', 'm9']:
    data[col] = data[col].map( {'F': 0, 'T': 1} ).astype(float)

# evaluate categorical features

In [11]:
categories_dict = {}
for cat in data.columns:
    if data[cat].dtypes == 'object':
        categories_dict[cat] = list(data[cat].unique())
        n_categories = data[cat].unique().size
        print("number of categories for variable '{}': {} ".format(cat, n_categories))

number of categories for variable 'productcd': 5 
number of categories for variable 'card4': 5 
number of categories for variable 'card6': 4 
number of categories for variable 'p_emaildomain': 58 
number of categories for variable 'r_emaildomain': 45 
number of categories for variable 'm4': 4 


# report data types

In [12]:
numerical_cols, other_cols = get_numerical_mask(data)
# numerical_cols.remove('y')
print(numerical_cols)

num_df = pd.DataFrame(numerical_cols)
other_df = pd.DataFrame(other_cols)

['y', 'transactiondt', 'transactionamt', 'card1', 'card2', 'card3', 'card5', 'addr1', 'addr2', 'dist1', 'dist2', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10', 'c11', 'c12', 'c13', 'c14', 'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8', 'd9', 'd10', 'd11', 'd12', 'd13', 'd14', 'd15', 'm1', 'm2', 'm3', 'm5', 'm6', 'm7', 'm8', 'm9']


#### concatenate

In [13]:
report_df = pd.concat([num_df,other_df], ignore_index=True, axis=1)
report_df.columns = ['numerical_cols', 'non_numerical_cols']

# Checking  for possible anomalies in the database
Here we check for some possible anomalies. If there was one, we should look for solutions such as: contact the team responsible for data engineering or anyone who has a good understanding of the provided data, search for flaws in the data pipeline, or at least document for those anomalies.

Check if there is any oddity in data.

In [14]:
data.loc[:, ].describe()

,y,transactiondt,transactionamt,card1,card2,card3,card5,addr1,addr2,dist1,...,d14,d15,m1,m2,m3,m5,m6,m7,m8,m9
count,10000.000000,1.000000e+04,10000.000000,10000.000000,9842.000000,9978.000000,9934.000000,8849.000000,8849.000000,4111.000000,...,1036.000000,8511.000000,5476.000000,5476.000000,5476.000000,4029.000000,7126.000000,4162.000000,4162.000000,4162.000000
mean,0.036300,7.378482e+06,137.002317,9879.690000,361.055375,153.336140,198.852829,289.112668,86.820093,118.680370,...,54.573359,164.542004,0.999817,0.892805,0.776662,0.441301,0.465900,0.129265,0.354877,0.840221
std,0.187045,4.599073e+06,247.033617,4917.656486,158.014321,11.351212,41.632274,101.642908,2.419210,363.370928,...,130.815766,203.495484,0.013514,0.309389,0.416521,0.496604,0.498871,0.335533,0.478534,0.366445
min,0.000000,8.841000e+04,0.272000,1001.000000,100.000000,100.000000,100.000000,110.000000,19.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,3.030122e+06,42.950000,6019.000000,206.000000,150.000000,166.000000,204.000000,87.000000,3.000000,...,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,0.000000,7.327499e+06,67.950000,9633.000000,361.000000,150.000000,226.000000,299.000000,87.000000,9.000000,...,0.000000,55.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,0.000000,1.123440e+07,122.000000,14276.000000,512.000000,150.000000,226.000000,330.000000,87.000000,25.000000,...,1.000000,313.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000
max,1.000000,1.581012e+07,4463.950000,18390.000000,600.000000,226.000000,237.000000,536.000000,96.000000,4966.000000,...,730.000000,807.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# save data

In [15]:
data.head()

,y,transactiondt,transactionamt,productcd,card1,card2,card3,card4,card5,card6,...,d15,m1,m2,m3,m4,m5,m6,m7,m8,m9
id,,,,,,,,,,,,,,,,,,,,,
3486774,0,13107389,38.056,C,9633,130.0,185.0,visa,138.0,debit,...,0.0,NaN,NaN,NaN,M2,NaN,NaN,NaN,NaN,NaN
3062695,0,1650884,150.000,R,15063,514.0,150.0,visa,226.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3273443,0,7048761,56.500,W,9006,555.0,143.0,mastercard,224.0,debit,...,2.0,1.0,1.0,1.0,M0,0.0,NaN,0.0,0.0,1.0
3384445,0,10011292,8.459,C,11201,103.0,185.0,visa,226.0,debit,...,0.0,NaN,NaN,NaN,M2,NaN,NaN,NaN,NaN,NaN
3489059,0,13159069,77.950,W,7919,194.0,150.0,mastercard,166.0,debit,...,0.0,1.0,0.0,0.0,NaN,NaN,1.0,0.0,0.0,0.0


In [16]:
data.to_csv(os.path.join(outputs, 'data.csv'))